In [5]:
import pandas as pd
import numpy as np
import pickle

import sys
sys.path.insert(0, "../src")
sys.path.insert(0, "../")
from d2v_recommender import *
from config import config
from src.processing import load_d2v_formated_data, turn_ratings_into_matches


In [153]:
raw_data_path = config.raw_data_path
train_data_path = config.train_data_path
test_data_path = config.test_data_path
max_u_id=100
turn_into_matches=True
MAX_ID_IN_DATASET = 130000
"""
Separate 20% of ratings from 100% of users.
:param nrows: use None for all
:return:
"""
assert max_u_id <= MAX_ID_IN_DATASET
print("Train-Test splitting")
ratings = pd.read_csv(raw_data_path, names=["rater", "rated", "r"])

# Shuffle to avoid bias of ordered rated users
ratings = ratings.sample(frac=1).reset_index(drop=True)

# list of all raters, already integers in our dataset
u = range(1, min(MAX_ID_IN_DATASET, max_u_id) + 1)
test = pd.DataFrame(columns=ratings.columns)
train = pd.DataFrame(columns=ratings.columns)
# TODO: coulb be optimized with a groupby ?
# for u_id in tqdm(u):
#     temp = ratings[ratings["rater"] == u_id]
#     n = len(temp)
#     train_size = int((1 - config.test_ratio) * n)

#     dummy_train = temp.iloc[:train_size]
#     dummy_test = temp.iloc[train_size:]

#     if turn_into_matches:
#         dummy_train, dummy_test = turn_ratings_into_matches(dummy_train, dummy_test)

#     # TODO: not efficint. Optimize!
#     test = pd.concat([test, dummy_test], ignore_index=True)
#     train = pd.concat([train, dummy_train], ignore_index=True)

Train-Test splitting


In [154]:
# ratings = pd.read_csv(raw_data_path, names=["rater", "rated", "r"])
ratings = ratings.sample(frac=1).reset_index(drop=True)
ratings

,rater,rated,r
0,62513,33216,10
1,93021,35845,7
2,22604,110484,1
3,119824,214650,6
4,24402,129239,2
...,...,...,...
17359341,8098,117364,8
17359342,74522,132293,10
17359343,31073,192933,10
17359344,18215,3629,7


In [159]:
from sklearn.model_selection import StratifiedShuffleSplit
# set train/test split
df = ratings[ratings["rater"]<=1000]
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
sss = sss.split(df, df["rater"])
train_idx, test_idx = list(sss)[0]
df_train = df.iloc[train_idx].set_index("rater")
df_test = df.iloc[test_idx].set_index("rater")

# get the quantile for the raters
quantiles = df.groupby(df["rater"])["r"].apply(lambda x: np.quantile(x, q=0.75)).to_frame().reset_index()  # df with rater as index and quantile as value
quantiles.columns = ["rater","r_quantile"]

In [156]:
quantiles

,rater,r_quantile
0,1,9.0
1,2,10.0
2,3,9.0
3,4,9.0
4,5,10.0
...,...,...
995,996,10.0
996,997,9.0
997,998,9.5
998,999,10.0


In [160]:
df_train = pd.merge(df_train, quantiles, left_on=df_train.index, right_on=quantiles.rater).drop(columns=["key_0"])
df_train["m"] = 1.0 * (df_train["r"]>=df_train["r_quantile"])
# df_train

df_test = pd.merge(df_test, quantiles, left_on=df_test.index, right_on=quantiles.rater).drop(columns=["key_0"])
df_test["m"] = 1.0 * (df_test["r"]>=df_test["r_quantile"])
df_test

,rated,r,rater,r_quantile,m
0,50580,1,456,4.00,0.0
1,78188,1,456,4.00,0.0
2,98294,6,456,4.00,1.0
3,207600,1,456,4.00,0.0
4,50904,1,456,4.00,0.0
...,...,...,...,...,...
14650,113621,2,747,8.75,0.0
14651,117544,4,232,9.75,0.0
14652,135132,10,232,9.75,1.0
14653,44437,5,635,8.00,0.0


In [158]:
max_u_id = 1000
df = ratings[ratings["rater"]<=max_u_id]
df["set"] = "train"
df["set"].values[-500:] = "test"
df["m"] = (df["r"].values >= df.groupby(df["rater"]).transform(lambda x: x[x["set"]=="train"]["r"].quantile(q=0.75)).values).astype(int)

/opt/conda/envs/d2v_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


KeyError: 'set'

In [90]:
df

,rater,rated,r,set
70,712,165790,3,train
97,325,179747,1,train
122,855,55139,3,train
300,29,96663,7,train
464,290,133987,10,train
...,...,...,...,...
17358740,100,33912,8,test
17358810,134,208884,6,test
17358924,349,100855,10,test
17358993,920,12916,1,test


In [50]:
df.max()

rater       100
rated    220953
r            10
dtype: int64

In [13]:
ratings.set_index("rater").r.groupby("rater").transform(lambda x: x.quantile(q=0.5))

KeyboardInterrupt: 